In [ ]:
import abipy
import numpy as np
from abipy import abilab
from abipy.dfpt.converters import phonopy_to_abinit, abinit_to_phonopy

import plotly
import plotly.graph_objects as go


## Quick test for abinit-phonopy-abinit conversion

In [ ]:
filename = 'data/reduced_cell/04_fcc100_dfpt_10/outdata/out_DDB'

ddb = abilab.abiopen(filename)
# phonopy_obj = ddb.anaget_phonopy_ifc(output_dir_path="output_dir")
phonopy_obj = ddb.anaget_phonopy_ifc()

In [ ]:
ddb_reconstructed = phonopy_to_abinit(
    unit_cell = ddb.structure,
    # unit_cell = phonopy_obj.unitcell.get_cell(),
    supercell_matrix = phonopy_obj.supercell_matrix, 
    # supercell_matrix = phonopy_obj.supercell.get_cell(), 
    out_ddb_path = "data/reduced_cell/04_fcc100_dfpt_10/outdata/test/out_DDB",
    force_constants= phonopy_obj.dynamical_matrix.get_force_constants(),
    qpt_list= ddb.qpoints.to_array()
    )

In [ ]:
for qpoint in ddb.qpoints:
    # print(qpoint)
    diff = ddb.computed_dynmat[qpoint] - ddb_reconstructed.computed_dynmat[qpoint]

    original_max_value = ddb.computed_dynmat[qpoint]["cvalue"].abs().max()

    max_diff = diff["cvalue"].abs().max()
    max_diff_inds = diff["cvalue"].abs().idxmax()
    original_at_max_diff = ddb.computed_dynmat[qpoint]["cvalue"].abs()[max_diff_inds]
    new_at_max_diff = ddb_reconstructed.computed_dynmat[qpoint]["cvalue"].abs()[max_diff_inds]
    print(f"max_diff: {max_diff}")
    print(f"original_at_max_diff: {original_at_max_diff}")
    print(f"new_at_max_diff: {new_at_max_diff}")
    print(f"percentage: {max_diff/original_at_max_diff*100}")
    print(f"original_max_value: {original_max_value}")
    print("")

## Phonopy 

In [ ]:
def lorentzian( x, x0, gamma):
    return   gamma**2 / (( x - x0 )**2 + gamma**2)
    # return  1/(np.pi * gamma) * gamma**2 / (( x - x0 )**2 + gamma**2)

In [ ]:
filename = 'data/reduced_cell/04_fcc100_dfpt_11/outdata/out_DDB'

ddb = abilab.abiopen(filename)
phonopy_obj = ddb.anaget_phonopy_ifc()


In [ ]:
ddb.qpoints

In [ ]:
freqs = phonopy_obj.get_frequencies([0., 0., 0.5])

omega = np.linspace(0,15,10001) # THz

f = np.zeros_like(omega)
for freq in freqs:
    f += lorentzian(omega, freq, 0.4)


fig = go.Figure(data=go.Scatter(x=omega, y=f))
fig.show()


In [ ]:
Nq = 11
Nf = 1001
gamma = 0.1

qpath = np.linspace(0, 1, Nq)
omega = np.linspace(0, 15, Nf) # THz

f = np.zeros((Nq, Nf))
for (i, qpt) in enumerate(qpath):
    freqs = phonopy_obj.get_frequencies([qpt, 0, 0])

    for freq in freqs:
        f[i,:] += lorentzian(omega, freq, gamma)

fig = go.Figure(data=go.Heatmap(
    z=f.transpose(),
    x=qpath,
    y=omega,
    colorscale='Viridis'))

# fig.show()

In [ ]:
Nq = 11
Nf = 1001
gamma = 0.1

qpath = np.linspace(0, 1, Nq)
omega = np.linspace(0, 15, Nf) # THz

f = np.zeros((Nq, Nf))
for (i, qpt) in enumerate(qpath):
    freqs = phonopy_obj.get_frequencies([0, 0, qpt])

    for freq in freqs:
        f[i,:] += lorentzian(omega, freq, gamma)

fig = go.Figure(data=go.Heatmap(
        z=f.transpose(),
        x=qpath,
        y=omega,
        colorscale='Viridis'))

# fig.show()

In [ ]:
# phonopy_obj.force_constants

In [ ]:
phonopy_obj.run_mesh()

In [ ]:
phonopy_obj.get_mesh()

In [ ]:
from phonopy.phonon.band_structure import get_band_qpoints_and_path_connections

path = [[[0, 0, 0], [0.5, 0, 0.5], [0.625, 0.25, 0.625]],
        [[0.375, 0.375, 0.75], [0, 0, 0], [0.5, 0.5, 0.5], [0.5, 0.25, 0.75]]]
labels = ["$\\Gamma$", "X", "U", "K", "$\\Gamma$", "L", "W"]
qpoints, connections = get_band_qpoints_and_path_connections(path, npoints=51)

phonopy_obj.run_band_structure(qpoints, path_connections=connections, labels=labels)
phonopy_obj.plot_band_structure().show()

In [ ]:
# phonopy_obj.run_mesh([20, 20, 1])
phonopy_obj.run_total_dos()

In [ ]:
# phonon.auto_projected_dos(plot=True).show()

phonopy_obj.run_mesh([20, 20, 1], with_eigenvectors=True, is_mesh_symmetry=False)

phonopy_obj.run_projected_dos()
phonopy_obj.plot_projected_dos().show()

In [ ]:

path=[
    [[0,x,0] for x in np.linspace(0,.5,101)],
    [[x,.5,0] for x in np.linspace(0,.5,101)],
    [[.5-x,.5-x,0] for x in np.linspace(0,.5,101)],
]
phonopy_obj.run_band_structure(path)
phonopy_obj.plot_band_structure_and_dos()

In [ ]:

path=[
    [[0,0,x] for x in np.linspace(0,.5,101)],
    [[x,x,.5-x] for x in np.linspace(0,.5,101)],
]
phonopy_obj.run_band_structure(path)
phonopy_obj.run_mesh([16, 16, 1])
phonopy_obj.run_total_dos()
phonopy_obj.plot_band_structure_and_dos()

In [ ]:
# phonopy_obj.auto_band_structure(plot=True).show()
phonopy_obj.auto_band_structure()
phonopy_obj.plot_band_structure_and_dos()

In [ ]:
phonopy_obj.unitcell.get_scaled_positions(
)

In [ ]:
s = phonopy_obj.get_supercell()

In [ ]:
ddb.qpoints

In [ ]:
# direction = np.array([1,0,0])
direction = np.array([0,0,1])

ifcs=phonopy_obj.get_force_constants()[0]
forces = np.dot(ifcs, direction)

magnitude = np.linalg.norm(forces, axis=1)[:,np.newaxis]
# forces *= np.log(magnitude)/magnitude

In [ ]:

inds = np.ravel_multi_index(np.triu_indices(4), (4,4))

data = []
for offset in inds:
    disp = s.positions[offset,0:2] - s.positions[0,0:2]
    data.append(go.Scatter(
        x=s.positions[offset::16,2], 
        y=magnitude[offset::16,0], 
        name = f"{np.round(disp,3)}"
    ))

fig = go.Figure(data)
fig.show()

In [ ]:
x, y, z = np.transpose(s.positions[::16])
# x, y, z = np.transpose(s.positions)
u, v, w = np.transpose(forces[::16])
# u, v, w = np.transpose(forces)


fig = go.Figure(data=[
    go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=np.linspace(0,1,len(z)),                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=1
        )
    ),
    go.Cone(
        x=x,
        y=y,
        z=z,
        u=u,
        v=v,
        w=w,
        sizemode="absolute",
        sizeref=5,
    )
    
])
# fig = go.Figure(data=[
# 
#     go.Cone(
#         x=x,
#         y=y,
#         z=z,
#         u=u,
#         v=v,
#         w=w,
#         sizemode="absolute",
#         sizeref=5,
#     )
#     
# ])



fig.layout.scene.camera.projection.type = "orthographic"
fig.layout.scene.aspectmode='data'
# fig.layout.scene.aspectratio=dict(x=1, y=1, z=1)

fig.show()

In [ ]:
magnitude[::16]

# Comparing with bulk